In [15]:
import pandas as pd
import glob
import os
import shutil

In [16]:
code_to_country = {
    'ENG': 'England',
    'ITA': 'Italy',
    'FRA': 'France',
    'SUI': 'Switzerland',
    'NGA': 'Nigeria',
    'CIV': 'Ivory Coast',
    'ESP': 'Spain',
    'ARG': 'Argentina',
    'SWE': 'Sweden',
    'BRA': 'Brazil',
    'ALG': 'Algeria',
    'AUT': 'Austria',
    'ALB': 'Albania',
    'IDN': 'Indonesia',
    'GER': 'Germany',
    'IRN': 'Iran',
    'MAR': 'Morocco',
    'CRO': 'Croatia',
    'URU': 'Uruguay',
    'NED': 'Netherlands',
    'ZAM': 'Zambia',
    'POR': 'Portugal',
    'CZE': 'Czech Republic',
    'POL': 'Poland',
    'DEN': 'Denmark',
    'GNB': 'Guinea-Bissau',
    'SVN': 'Slovenia',
    'MLI': 'Mali',
    'NOR': 'Norway',
    'ROU': 'Romania',
    'TUN': 'Tunisia',
    'CAN': 'Canada',
    'CPV': 'Cape Verde',
    'NZL': 'New Zealand',
    'TUR': 'Turkey',
    'SEN': 'Senegal',
    'COL': 'Colombia',
    'BEL': 'Belgium',
    'MTQ': 'Martinique',
    'SCO': 'Scotland',
    'SVK': 'Slovakia',
    'GHA': 'Ghana',
    'IRL': 'Republic of Ireland',
    'CMR': 'Cameroon',
    'MKD': 'North Macedonia',
    'CHI': 'Chile',
    'LTU': 'Lithuania',
    'ISL': 'Iceland',
    'SUR': 'Suriname',
    'GRE': 'Greece',
    'SRB': 'Serbia',
    'SLE': 'Sierra Leone',
    'JPN': 'Japan',
    'CYP': 'Cyprus',
    'BIH': 'Bosnia and Herzegovina',
    'UKR': 'Ukraine',
    'MNE': 'Montenegro',
    'GEO': 'Georgia',
    'PER': 'Peru',
    'ANG': 'Angola',
    'CGO': 'Republic of the Congo',
    'USA': 'United States',
    'RUS': 'Russia',
    'ARM': 'Armenia',
    'EQG': 'Equatorial Guinea',
    'MEX': 'Mexico',
    'GAB': 'Gabon',
    'KVX': 'Kosovo',
    'PAR': 'Paraguay',
    'UZB': 'Uzbekistan',
    'JAM': 'Jamaica',
    'ISR': 'Israel',
    'ZIM': 'Zimbabwe'
}

In [17]:
# Step 1: Backup raw data
shutil.copytree("../data/raw", "../data/raw_backup_12_06_25", dirs_exist_ok=True)
print("✅ Backup complete!")

✅ Backup complete!


In [18]:
# Step 0: Utility function to clean all column names
def clean_columns(df):
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(" ", "_")
                  .str.replace("-", "_")
                  .str.replace(".", "_", regex=False)
    )
    return df

# Step 1: Define the rename map based on cleaned columns
rename_map = {
    "rk": "rank",
    "nation": "nationality",
    "gls_1": "goals_per90",
    "ast_1": "assists_per90",
    "g+a_1": "goals_and_assists_per90",
    "g_pk_1": "non_penalty_goals_per90",
    "g+a_pk": "non_penalty_goals_and_assists_per90",
    "pos": "position",
    "squad": "team",
    "born": "birth_year",
    "min": "minutes",
    "gls": "goals",
    "ast": "assists",
    "pk": "pens_scored",
    "pkatt": "pens_attempted",
    "g+a": "goals_and_assists",
    "g_pk": "non_penalty_goals",
    "g+a_pk": "non_penalty_goals_and_assists",
    "fls": "fouls_committed",
    "fld": "fouls_drawn",
    "mp": "matches_played",
    "off": "offsides",
    "prgc": "progressive_carries",
    "prgr": "progressive_receptions",
    "recov": "ball_recoveries",
    "won": "aerial_duels_won",
    "lost": "aerial_duels_lost",
    "won%": "aerial_duel_win_rate",
    "xag": "xag_expected_assisted_goals",
    "xg": "xg_expected_goals",
    "npxg": "npxg_non_penalty_xg",
    "xa": "xa_expected_assists",
}

# Step 2: Define columns to delete
cols_to_delete = [
    "rank", "matches", "crdy", "crdr", "90s", "pkcon", "og", "int", "tklw", "crs",
    "2crdy", "progressive_receptions", "ball_recoveries",
    "aerial_duels_won", "aerial_duels_lost", "aerial_duel_win_rate", "fouls_committed", "xg_1", "xag_1", "xg+xag", "npxg_1", "npxg+xag_1"
]

In [19]:
# Step 3: Find all raw CSVs
csv_files = glob.glob("../data/raw/**/*.csv", recursive=True)
print(f"Found {len(csv_files)} raw CSVs.")

Found 106 raw CSVs.


In [20]:
# Step 4: Clean each file
for idx, file in enumerate(csv_files):
    print(f"Cleaning file {idx+1}/{len(csv_files)}: {file}")

    # Read CSV
    df = pd.read_csv(file)

    # 🧹 CLEAN columns first
    df = clean_columns(df)

    # 🛠 Then RENAME
    df = df.rename(columns=rename_map)

    # 🌍 Fix nationality values
    df['country_code'] = df['nationality'].apply(lambda x: x.split()[-1] if isinstance(x, str) else None)
    df['nationality'] = df['country_code'].map(code_to_country)

    # 🧹 Drop unwanted columns
    df = df.drop(columns=[col for col in cols_to_delete if col in df.columns], errors='ignore')

    # 🧹 Drop any remaining unnamed columns
    df = df.drop(columns=[col for col in df.columns if col.startswith("unnamed")], errors='ignore')

    # Save cleaned file
    clean_path = file.replace("raw", "clean")
    print("→ Columns after cleaning:", df.columns.tolist())
    print("→ Sample nationality values:", df['nationality'].unique()[:5] if 'nationality' in df.columns else "N/A")
    print("→ Saving to:", clean_path)
    os.makedirs(os.path.dirname(clean_path), exist_ok=True)
    df.to_csv(clean_path, index=False)

print("✅ All files cleaned and saved into the clean/ folder!")

Cleaning file 1/106: ../data/raw\bundesliga\Bundesliga_2016-2017_misc.csv
→ Columns after cleaning: ['player', 'nationality', 'position', 'team', 'age', 'birth_year', 'fouls_drawn', 'offsides', 'pkwon', 'country_code']
→ Sample nationality values: ['Argentina' 'Albania' 'Germany' 'Austria' 'Spain']
→ Saving to: ../data/clean\bundesliga\Bundesliga_2016-2017_misc.csv
Cleaning file 2/106: ../data/raw\bundesliga\Bundesliga_2016-2017_standard.csv
→ Columns after cleaning: ['player', 'nationality', 'position', 'team', 'age', 'birth_year', 'matches_played', 'starts', 'minutes', 'goals', 'assists', 'goals_and_assists', 'non_penalty_goals', 'pens_scored', 'pens_attempted', 'goals_per90', 'assists_per90', 'goals_and_assists_per90', 'non_penalty_goals_per90', 'non_penalty_goals_and_assists', 'country_code']
→ Sample nationality values: ['Argentina' 'Albania' 'Germany' 'Austria' 'Spain']
→ Saving to: ../data/clean\bundesliga\Bundesliga_2016-2017_standard.csv
Cleaning file 3/106: ../data/raw\bundes

In [21]:
# drop duplicate columns based on suffix
# fix nationality column
# rename goals + assists per 90 columns
#reorder columns
#new_order = ['player', 'age', 'team', 'position', 'minutes', 'nationality_clean', 'goals', 'assists']
#df = df[new_order]
#or
#priority_cols = ['player', 'team', 'nationality_clean']
#df = df[priority_cols + [col for col in df.columns if col not in priority_cols]]

# change age and year data types
#big5_standard_misc.

# Step 2: Define the rename map
#rename_map_post_merge = {
  
#}